
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Lab - Creating Bronze Tables from JSON Files
### Duration: ~ 15 minutes

In this lab you will ingest a JSON file as Delta table and then flatten the JSON formatted string column.

### Learning Objectives
  - Inspect a raw JSON file.
  - Read in JSON files to a Delta table and flatten the JSON formatted string column.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default and you have a Shared SQL warehouse.

<!-- ![Select Cluster](./Includes/images/selecting_cluster_info.png) -->

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.


## A. Classroom Setup

Run the following cell to configure your working environment for this notebook.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

In [0]:
%run ./Includes/Classroom-Setup-07L

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


----------------------------------------------------------------------------------------
Directory /Volumes/dbacademy/ops/labuser11169727_1755226239@vocareum_com/csv_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser11169727_1755226239@vocareum_com/json_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser11169727_1755226239@vocareum_com/xml_demo_files already exists. No action taken.
----------------------------------------------------------------------------------------



Generated file: /Volumes/dbacademy/ops/labuser11169727_1755226239@vocareum_com/json_demo_files/lab_kafka_events.json


File written with 1 malformed 'timestamp' at position 64: /Volumes/dbacademy/ops/labuser11169727_1755226239@vocareum_com/json_demo_files/lab_kafka_events_challenge.json


Run the cell below to view your default catalog and schema. Notice that your default catalog is **dbacademy** and your default schema is your unique **labuser** schema.

**NOTE:** The default catalog and schema are pre-configured for you to avoid the need to specify the three-level name when writing your tables (i.e., catalog.schema.table).

In [0]:
SELECT current_catalog(), current_schema()

current_catalog(),current_schema()
dbacademy,labuser11169727_1755226239


## B. Lab - JSON Ingestion
**Scenario:** You are working with your data team on ingesting a JSON file into Databricks. Your job is to ingest the JSON file as is into a bronze table, then create a second bronze table that flattens the JSON formatted string column in the raw bronze table for downstream processing.

### B1. Inspect the Dataset

1. In the cell below, view the value of the Python variable `DA.paths.working_dir`. This variable will reference your **dbacademy.ops.labuser** volume, as each user has a different source volume. This variable is created within the classroom setup script to dynamically reference your unique volume.

   Run the cell and review the results. You’ll notice that the `DA.paths.working_dir` variable points to your `/Volumes/dbacademy/ops/labuser` volume.

**Note:** Instead of using the `DA.paths.working_dir` variable, you could also specify the path name directly by right clicking on your volume and selecting **Copy volume path**.

In [0]:
%python
print(DA.paths.working_dir)

/Volumes/dbacademy/ops/labuser11169727_1755226239@vocareum_com


Run the cell to view the data in the `/Volumes/dbacademy/ops/your-labuser-name/json_demo_files/lab_kafka_events.json` file in the location from above.

In [0]:
%python
spark.sql(f'''
          SELECT * 
          FROM json.`{DA.paths.working_dir}/json_demo_files/lab_kafka_events.json`
          ''').display()

key,timestamp,value
event_0,1.755227997685932E9,eyJ1c2VyX2lkIjogInVzZXJfOTQ0MiIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDgtMTVUMDM6MTk6NTcuNjg1ODg1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzE3NSIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAyNi4zNH0sIHsiaXRlbV9pZCI6ICJpdGVtXzMwOCIsICJxdWFudGl0eSI6IDIsICJwcmljZV91c2QiOiAzOC40Mn1dfQ==
event_1,1.755227997685954E9,eyJ1c2VyX2lkIjogInVzZXJfNjIwMiIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTkzOCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV85MDUiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMTMuMDR9XX0=
event_2,1.75522799768597E9,eyJ1c2VyX2lkIjogInVzZXJfMTg1NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTk1NyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV81NjYiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogNTEuNjN9LCB7Iml0ZW1faWQiOiAiaXRlbV8zMTAiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMTYuOTl9XX0=
event_3,1.755227997685985E9,eyJ1c2VyX2lkIjogInVzZXJfOTQzMCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTk3MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV81NjYiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogMzcuMDN9LCB7Iml0ZW1faWQiOiAiaXRlbV80MzYiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogODUuMzZ9XX0=
event_4,1.755227997685996E9,eyJ1c2VyX2lkIjogInVzZXJfNzc4NiIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTk4NyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8zNjYiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogNDYuMDl9XX0=
event_5,1.755227997686006E9,eyJ1c2VyX2lkIjogInVzZXJfNDE3NSIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTk5OCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV80NjAiLCAicXVhbnRpdHkiOiA0LCAicHJpY2VfdXNkIjogMjUuOTN9XX0=
event_6,1.755227997686019E9,eyJ1c2VyX2lkIjogInVzZXJfMjE1OSIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NjAwOCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8zNjYiLCAicXVhbnRpdHkiOiA0LCAicHJpY2VfdXNkIjogNTcuNDN9LCB7Iml0ZW1faWQiOiAiaXRlbV8xMzIiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogNjQuNTR9XX0=
event_7,1.755227997686036E9,eyJ1c2VyX2lkIjogInVzZXJfNzM4NiIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NjAyMSIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV84OTQiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogNDguMDd9LCB7Iml0ZW1faWQiOiAiaXRlbV83MjYiLCAicXVhbnRpdHkiOiA0LCAicHJpY2VfdXNkIjogMzguMjd9LCB7Iml0ZW1faWQiOiAiaXRlbV83NzkiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogODMuMTV9XX0=
event_8,1.755227997686053E9,eyJ1c2VyX2lkIjogInVzZXJfODg1NyIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NjAzOSIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV82ODMiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogMjMuMDF9LCB7Iml0ZW1faWQiOiAiaXRlbV81MjciLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMjMuMjV9LCB7Iml0ZW1faWQiOiAiaXRlbV83NjAiLCAicXVhbnRpdHkiOiA0LCAicHJpY2VfdXNkIjogNDQuNjV9XX0=
event_9,1.755227997686064E9,eyJ1c2VyX2lkIjogInVzZXJfOTgxOSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDgtMTVUMDM6MTk6NTcuNjg2MDU1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzE1NyIsICJxdWFudGl0eSI6IDIsICJwcmljZV91c2QiOiA2Ny45MX1dfQ==


### B2. Create the Raw Bronze Table

Inspect and run the code below to ingest the raw JSON file `/Volumes/dbacademy/ops/your-labuser-name/json_demo_files/lab_kafka_events.json` and create the **lab7_lab_kafka_events_raw** table.

Notice the following:
- The **value** column is decoded.
- The **decoded_value** column was created and returns the decoded column as a JSON-formatted string.


In [0]:
CREATE OR REPLACE TABLE lab7_lab_kafka_events_raw
AS
SELECT 
  *,
  cast(unbase64(value) as STRING) as decoded_value
FROM read_files(
        DA.paths_working_dir || '/json_demo_files/lab_kafka_events.json',
        format => "json", 
        schema => '''
          key STRING, 
          timestamp DOUBLE, 
          value STRING
        ''',
        rescueddatacolumn => '_rescued_data'
      );

-- View the table
SELECT *
FROM lab7_lab_kafka_events_raw;

key,timestamp,value,_rescued_data,decoded_value
event_0,1.755227997685932E9,eyJ1c2VyX2lkIjogInVzZXJfOTQ0MiIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDgtMTVUMDM6MTk6NTcuNjg1ODg1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzE3NSIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAyNi4zNH0sIHsiaXRlbV9pZCI6ICJpdGVtXzMwOCIsICJxdWFudGl0eSI6IDIsICJwcmljZV91c2QiOiAzOC40Mn1dfQ==,null,"{""user_id"": ""user_9442"", ""event_type"": ""view"", ""event_timestamp"": ""2025-08-15T03:19:57.685885"", ""items"": [{""item_id"": ""item_175"", ""quantity"": 3, ""price_usd"": 26.34}, {""item_id"": ""item_308"", ""quantity"": 2, ""price_usd"": 38.42}]}"
event_1,1.755227997685954E9,eyJ1c2VyX2lkIjogInVzZXJfNjIwMiIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTkzOCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV85MDUiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMTMuMDR9XX0=,null,"{""user_id"": ""user_6202"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-08-15T03:19:57.685938"", ""items"": [{""item_id"": ""item_905"", ""quantity"": 1, ""price_usd"": 13.04}]}"
event_2,1.75522799768597E9,eyJ1c2VyX2lkIjogInVzZXJfMTg1NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTk1NyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV81NjYiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogNTEuNjN9LCB7Iml0ZW1faWQiOiAiaXRlbV8zMTAiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMTYuOTl9XX0=,null,"{""user_id"": ""user_1854"", ""event_type"": ""click"", ""event_timestamp"": ""2025-08-15T03:19:57.685957"", ""items"": [{""item_id"": ""item_566"", ""quantity"": 1, ""price_usd"": 51.63}, {""item_id"": ""item_310"", ""quantity"": 2, ""price_usd"": 16.99}]}"
event_3,1.755227997685985E9,eyJ1c2VyX2lkIjogInVzZXJfOTQzMCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTk3MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV81NjYiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogMzcuMDN9LCB7Iml0ZW1faWQiOiAiaXRlbV80MzYiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogODUuMzZ9XX0=,null,"{""user_id"": ""user_9430"", ""event_type"": ""click"", ""event_timestamp"": ""2025-08-15T03:19:57.685973"", ""items"": [{""item_id"": ""item_566"", ""quantity"": 3, ""price_usd"": 37.03}, {""item_id"": ""item_436"", ""quantity"": 5, ""price_usd"": 85.36}]}"
event_4,1.755227997685996E9,eyJ1c2VyX2lkIjogInVzZXJfNzc4NiIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTk4NyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8zNjYiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogNDYuMDl9XX0=,null,"{""user_id"": ""user_7786"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-08-15T03:19:57.685987"", ""items"": [{""item_id"": ""item_366"", ""quantity"": 1, ""price_usd"": 46.09}]}"
event_5,1.755227997686006E9,eyJ1c2VyX2lkIjogInVzZXJfNDE3NSIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NTk5OCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV80NjAiLCAicXVhbnRpdHkiOiA0LCAicHJpY2VfdXNkIjogMjUuOTN9XX0=,null,"{""user_id"": ""user_4175"", ""event_type"": ""click"", ""event_timestamp"": ""2025-08-15T03:19:57.685998"", ""items"": [{""item_id"": ""item_460"", ""quantity"": 4, ""price_usd"": 25.93}]}"
event_6,1.755227997686019E9,eyJ1c2VyX2lkIjogInVzZXJfMjE1OSIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NjAwOCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8zNjYiLCAicXVhbnRpdHkiOiA0LCAicHJpY2VfdXNkIjogNTcuNDN9LCB7Iml0ZW1faWQiOiAiaXRlbV8xMzIiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogNjQuNTR9XX0=,null,"{""user_id"": ""user_2159"", ""event_type"": ""click"", ""event_timestamp"": ""2025-08-15T03:19:57.686008"", ""items"": [{""item_id"": ""item_366"", ""quantity"": 4, ""price_usd"": 57.43}, {""item_id"": ""item_132"", ""quantity"": 5, ""price_usd"": 64.54}]}"
event_7,1.755227997686036E9,eyJ1c2VyX2lkIjogInVzZXJfNzM4NiIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA4LTE1VDAzOjE5OjU3LjY4NjAyMSIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV84OT

### B3. Create the Flattened Bronze Table

1. Your goal is to flatten the JSON formatted string column **decoded_value** from the table **lab7_lab_kafka_events_raw** to create a new table named **lab7_lab_kafka_events_flattened** for downstream processing. The table should contain the following columns:
    - **key**
    - **timestamp**
    - **user_id**
    - **event_type**
    - **event_timestamp**
    - **items**

    You can use whichever technique you prefer:

    - Parse the JSON formatted string (easiest) to flatten
      - [Query JSON strings](https://docs.databricks.com/aws/en/semi-structured/json):

    - Convert the JSON formatted string as a VARIANT and flatten
      - [parse_json function](https://docs.databricks.com/gcp/en/sql/language-manual/functions/parse_json)

    - Convert the JSON formatted string to a STRUCT and flatten
      - [schema_of_json function](https://docs.databricks.com/aws/en/sql/language-manual/functions/schema_of_json)
      - [from_json function](https://docs.databricks.com/gcp/en/sql/language-manual/functions/from_json)

**NOTE:** View the lab solution notebook to view the solutions for each.

2. To begin, run the code below to view the final solution table **lab7_lab_kafka_events_flattened_solution**. This will give you an idea of what your final table should look like.

  **NOTE**: Depending on your solution, the data types of the columns may vary slightly.  


##### Optional Challenge

  As a challenge, after flattening the table, try converting the data types accordingly. Depending on your skill set, you may not convert all columns to the correct data types within the allotted time.

  - **key** STRING
  - **timestamp** DOUBLE
  - **user_id** STRING
  - **event_type** STRING
  - **event_timestamp** TIMESTAMP
  - **items** (STRUCT or VARIANT) depending on the method you used.

In [0]:
SELECT *
FROM lab7_lab_kafka_events_flattened_solution

key,timestamp,user_id,event_type,event_timestamp,items
event_0,1.755227997685932E9,user_9442,view,2025-08-15T03:19:57.685885Z,"List(List(item_175, 26.34, 3), List(item_308, 38.42, 2))"
event_1,1.755227997685954E9,user_6202,purchase,2025-08-15T03:19:57.685938Z,"List(List(item_905, 13.04, 1))"
event_2,1.75522799768597E9,user_1854,click,2025-08-15T03:19:57.685957Z,"List(List(item_566, 51.63, 1), List(item_310, 16.99, 2))"
event_3,1.755227997685985E9,user_9430,click,2025-08-15T03:19:57.685973Z,"List(List(item_566, 37.03, 3), List(item_436, 85.36, 5))"
event_4,1.755227997685996E9,user_7786,purchase,2025-08-15T03:19:57.685987Z,"List(List(item_366, 46.09, 1))"
event_5,1.755227997686006E9,user_4175,click,2025-08-15T03:19:57.685998Z,"List(List(item_460, 25.93, 4))"
event_6,1.755227997686019E9,user_2159,click,2025-08-15T03:19:57.686008Z,"List(List(item_366, 57.43, 4), List(item_132, 64.54, 5))"
event_7,1.755227997686036E9,user_7386,click,2025-08-15T03:19:57.686021Z,"List(List(item_894, 48.07, 2), List(item_726, 38.27, 4), List(item_779, 83.15, 1))"
event_8,1.755227997686053E9,user_8857,click,2025-08-15T03:19:57.686039Z,"List(List(item_683, 23.01, 5), List(item_527, 23.25, 1), List(item_760, 44.65, 4))"
event_9,1.755227997686064E9,user_9819,view,2025-08-15T03:19:57.686055Z,"List(List(item_157, 67.91, 2))"


3. Write the query in the cell below to read the **lab_kafka_events_raw** table and create the flattened table **lab7_lab_kafka_events_flattened** following the requirements from above.

In [0]:
---- Parse the JSON formatted STRING
CREATE OR REPLACE TABLE lab7_lab_kafka_events_flattened_str
AS
SELECT 
  key,
  timestamp,
  decoded_value:user_id,
  decoded_value:event_type,
  cast(decoded_value:event_timestamp AS TIMESTAMP),
  from_json(decoded_value:items,'ARRAY<STRUCT<item_id: STRING, price_usd: DOUBLE, quantity: BIGINT>>') AS items
FROM lab7_lab_kafka_events_raw;


---- Display the table
SELECT *
FROM lab7_lab_kafka_events_flattened_str;

key,timestamp,user_id,event_type,event_timestamp,items
event_0,1.755227997685932E9,user_9442,view,2025-08-15T03:19:57.685885Z,"List(List(item_175, 26.34, 3), List(item_308, 38.42, 2))"
event_1,1.755227997685954E9,user_6202,purchase,2025-08-15T03:19:57.685938Z,"List(List(item_905, 13.04, 1))"
event_2,1.75522799768597E9,user_1854,click,2025-08-15T03:19:57.685957Z,"List(List(item_566, 51.63, 1), List(item_310, 16.99, 2))"
event_3,1.755227997685985E9,user_9430,click,2025-08-15T03:19:57.685973Z,"List(List(item_566, 37.03, 3), List(item_436, 85.36, 5))"
event_4,1.755227997685996E9,user_7786,purchase,2025-08-15T03:19:57.685987Z,"List(List(item_366, 46.09, 1))"
event_5,1.755227997686006E9,user_4175,click,2025-08-15T03:19:57.685998Z,"List(List(item_460, 25.93, 4))"
event_6,1.755227997686019E9,user_2159,click,2025-08-15T03:19:57.686008Z,"List(List(item_366, 57.43, 4), List(item_132, 64.54, 5))"
event_7,1.755227997686036E9,user_7386,click,2025-08-15T03:19:57.686021Z,"List(List(item_894, 48.07, 2), List(item_726, 38.27, 4), List(item_779, 83.15, 1))"
event_8,1.755227997686053E9,user_8857,click,2025-08-15T03:19:57.686039Z,"List(List(item_683, 23.01, 5), List(item_527, 23.25, 1), List(item_760, 44.65, 4))"
event_9,1.755227997686064E9,user_9819,view,2025-08-15T03:19:57.686055Z,"List(List(item_157, 67.91, 2))"


In [0]:
---- Convert the JSON formatted string as a VARIANT
---- NOTE: The VARIANT decoded_value_variant column is included in this solution to display the column
---- NOTE: Variant data type will not work on Serverless Version 1.
CREATE OR REPLACE TABLE lab7_lab_kafka_events_flattened_variant
AS
SELECT
  key,
  timestamp,
  parse_json(decoded_value) AS decoded_value_variant,
  cast(decoded_value_variant:user_id AS STRING),
  decoded_value_variant:event_type :: STRING,
  decoded_value_variant:event_timestamp :: TIMESTAMP,
  decoded_value_variant:items
FROM lab7_lab_kafka_events_raw;


---- Display the table
SELECT *
FROM lab7_lab_kafka_events_flattened_variant;

key,timestamp,decoded_value_variant,user_id,event_type,event_timestamp,items
event_0,1.755227997685932E9,"{""event_timestamp"":""2025-08-15T03:19:57.685885"",""event_type"":""view"",""items"":[{""item_id"":""item_175"",""price_usd"":26.34,""quantity"":3},{""item_id"":""item_308"",""price_usd"":38.42,""quantity"":2}],""user_id"":""user_9442""}",user_9442,view,2025-08-15T03:19:57.685885Z,"[{""item_id"":""item_175"",""price_usd"":26.34,""quantity"":3},{""item_id"":""item_308"",""price_usd"":38.42,""quantity"":2}]"
event_1,1.755227997685954E9,"{""event_timestamp"":""2025-08-15T03:19:57.685938"",""event_type"":""purchase"",""items"":[{""item_id"":""item_905"",""price_usd"":13.04,""quantity"":1}],""user_id"":""user_6202""}",user_6202,purchase,2025-08-15T03:19:57.685938Z,"[{""item_id"":""item_905"",""price_usd"":13.04,""quantity"":1}]"
event_2,1.75522799768597E9,"{""event_timestamp"":""2025-08-15T03:19:57.685957"",""event_type"":""click"",""items"":[{""item_id"":""item_566"",""price_usd"":51.63,""quantity"":1},{""item_id"":""item_310"",""price_usd"":16.99,""quantity"":2}],""user_id"":""user_1854""}",user_1854,click,2025-08-15T03:19:57.685957Z,"[{""item_id"":""item_566"",""price_usd"":51.63,""quantity"":1},{""item_id"":""item_310"",""price_usd"":16.99,""quantity"":2}]"
event_3,1.755227997685985E9,"{""event_timestamp"":""2025-08-15T03:19:57.685973"",""event_type"":""click"",""items"":[{""item_id"":""item_566"",""price_usd"":37.03,""quantity"":3},{""item_id"":""item_436"",""price_usd"":85.36,""quantity"":5}],""user_id"":""user_9430""}",user_9430,click,2025-08-15T03:19:57.685973Z,"[{""item_id"":""item_566"",""price_usd"":37.03,""quantity"":3},{""item_id"":""item_436"",""price_usd"":85.36,""quantity"":5}]"
event_4,1.755227997685996E9,"{""event_timestamp"":""2025-08-15T03:19:57.685987"",""event_type"":""purchase"",""items"":[{""item_id"":""item_366"",""price_usd"":46.09,""quantity"":1}],""user_id"":""user_7786""}",user_7786,purchase,2025-08-15T03:19:57.685987Z,"[{""item_id"":""item_366"",""price_usd"":46.09,""quantity"":1}]"
event_5,1.755227997686006E9,"{""event_timestamp"":""2025-08-15T03:19:57.685998"",""event_type"":""click"",""items"":[{""item_id"":""item_460"",""price_usd"":25.93,""quantity"":4}],""user_id"":""user_4175""}",user_4175,click,2025-08-15T03:19:57.685998Z,"[{""item_id"":""item_460"",""price_usd"":25.93,""quantity"":4}]"
event_6,1.755227997686019E9,"{""event_timestamp"":""2025-08-15T03:19:57.686008"",""event_type"":""click"",""items"":[{""item_id"":""item_366"",""price_usd"":57.43,""quantity"":4},{""item_id"":""item_132"",""price_usd"":64.54,""quantity"":5}],""user_id"":""user_2159""}",user_2159,click,2025-08-15T03:19:57.686008Z,"[{""item_id"":""item_366"",""price_usd"":57.43,""quantity"":4},{""item_id"":""item_132"",""price_usd"":64.54,""quantity"":5}]"
event_7,1.755227997686036E9,"{""event_timestamp"":""2025-08-15T03:19:57.686021"",""event_type"":""click"",""items"":[{""item_id"":""item_894"",""price_usd"":48.07,""quantity"":2},{""item_id"":""item_726"",""price_usd"":38.27,""quantity"":4},{""item_id"":""item_779"",""price_usd"":83.15,""quantity"":1}],""user_id"":""user_7386""}",user_7386,click,2025-08-15T03:19:57.686021Z,"[{""item_id"":""item_894"",""price_usd"":48.07,""quantity"":2},{""item_id"":""item_726"",""price_usd"":38.27,""quantity"":4},{""item_id"":""item_779"",""price_usd"":83.15,""quantity"":1}]"
event_8,1.755227997686053E9,"{""event_timestamp"":""2025-08-15T03:19:57.686039"",""event_type"":""click"",""items"":[{""item_id"":""item_683"",""price_usd"":23.01,""quantity"":5},{""item_id"":""item_527"",""price_usd"":23.25,""quantity"":1},{""item_id"":""item_760"",""price_usd"":44.65,""quantity"":4}],""user_id"":""user_8857""}",user_8857,click,2025-08-15T03:19:57.686039Z,"[{""item_id"":""item_683"",""price_usd"":23.01,""quantity"":5},{""item_id"":""item_527"",""price_usd"":23.25,""quantity"":1},{""item_id"":""item_760"",""price_usd"":44.65,""quantity"":4}]"
event_9,1.755227997686064E9,"{""event_timestamp"":""2025-08-

In [0]:
---- Convert the JSON formatted string as a STRUCT

---- Return the structure of the JSON formatted string
SELECT schema_of_json(decoded_value)
FROM lab7_lab_kafka_events_raw
LIMIT 1;


---- Use the JSON structure from above within the from_json function to convert the JSON formatted string to a STRUCT
---- NOTE: The STRUCT decoded_value_struct column is included in this solution to display the column
CREATE OR REPLACE TABLE lab7_lab_kafka_events_flattened_struct
AS
SELECT
  key,
  timestamp,
  from_json(decoded_value, 'STRUCT<event_timestamp: STRING, event_type: STRING, items: ARRAY<STRUCT<item_id: STRING, price_usd: DOUBLE, quantity: BIGINT>>, user_id: STRING>') AS decoded_value_struct,
  decoded_value_struct.user_id,
  decoded_value_struct.event_type,
  cast(decoded_value_struct.event_timestamp AS TIMESTAMP),
  decoded_value_struct.items
FROM lab7_lab_kafka_events_raw;


---- Display the table
SELECT *
FROM lab7_lab_kafka_events_flattened_struct;

key,timestamp,decoded_value_struct,user_id,event_type,event_timestamp,items
event_0,1.755227997685932E9,"List(2025-08-15T03:19:57.685885, view, List(List(item_175, 26.34, 3), List(item_308, 38.42, 2)), user_9442)",user_9442,view,2025-08-15T03:19:57.685885Z,"List(List(item_175, 26.34, 3), List(item_308, 38.42, 2))"
event_1,1.755227997685954E9,"List(2025-08-15T03:19:57.685938, purchase, List(List(item_905, 13.04, 1)), user_6202)",user_6202,purchase,2025-08-15T03:19:57.685938Z,"List(List(item_905, 13.04, 1))"
event_2,1.75522799768597E9,"List(2025-08-15T03:19:57.685957, click, List(List(item_566, 51.63, 1), List(item_310, 16.99, 2)), user_1854)",user_1854,click,2025-08-15T03:19:57.685957Z,"List(List(item_566, 51.63, 1), List(item_310, 16.99, 2))"
event_3,1.755227997685985E9,"List(2025-08-15T03:19:57.685973, click, List(List(item_566, 37.03, 3), List(item_436, 85.36, 5)), user_9430)",user_9430,click,2025-08-15T03:19:57.685973Z,"List(List(item_566, 37.03, 3), List(item_436, 85.36, 5))"
event_4,1.755227997685996E9,"List(2025-08-15T03:19:57.685987, purchase, List(List(item_366, 46.09, 1)), user_7786)",user_7786,purchase,2025-08-15T03:19:57.685987Z,"List(List(item_366, 46.09, 1))"
event_5,1.755227997686006E9,"List(2025-08-15T03:19:57.685998, click, List(List(item_460, 25.93, 4)), user_4175)",user_4175,click,2025-08-15T03:19:57.685998Z,"List(List(item_460, 25.93, 4))"
event_6,1.755227997686019E9,"List(2025-08-15T03:19:57.686008, click, List(List(item_366, 57.43, 4), List(item_132, 64.54, 5)), user_2159)",user_2159,click,2025-08-15T03:19:57.686008Z,"List(List(item_366, 57.43, 4), List(item_132, 64.54, 5))"
event_7,1.755227997686036E9,"List(2025-08-15T03:19:57.686021, click, List(List(item_894, 48.07, 2), List(item_726, 38.27, 4), List(item_779, 83.15, 1)), user_7386)",user_7386,click,2025-08-15T03:19:57.686021Z,"List(List(item_894, 48.07, 2), List(item_726, 38.27, 4), List(item_779, 83.15, 1))"
event_8,1.755227997686053E9,"List(2025-08-15T03:19:57.686039, click, List(List(item_683, 23.01, 5), List(item_527, 23.25, 1), List(item_760, 44.65, 4)), user_8857)",user_8857,click,2025-08-15T03:19:57.686039Z,"List(List(item_683, 23.01, 5), List(item_527, 23.25, 1), List(item_760, 44.65, 4))"
event_9,1.755227997686064E9,"List(2025-08-15T03:19:57.686055, view, List(List(item_157, 67.91, 2)), user_9819)",user_9819,view,2025-08-15T03:19:57.686055Z,"List(List(item_157, 67.91, 2))"



&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>